In [ ]:
# Import library.
from PIL import Image, ImageOps
from tensorflow.keras.models import load_model
import cv2
import tensorflow
import keras
import numpy as np
import os

In [ ]:
# Load face classifier model.
face_cascade = 'haarcascade_frontalface_default.xml'
face_classifier = cv2.CascadeClassifier(face_cascade)

In [ ]:
# Load our face mask classifier model.
model = load_model("model_mask_detection2.h5")

In [ ]:
# Setting size 
size = (224, 224)
np.set_printoptions(suppress = True)

In [ ]:
# Choose camera
webcam = cv2.VideoCapture(0)
count = 0

# Open camera
while True:  
    # Capture image from camera into image_bgr.
    success, image_bgr = webcam.read()
    image_org = image_bgr.copy()
    
    # Adjust image color to be black and white.
    image_bw = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    
    # Adjust image color to rgb color.
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    
    # Detect face in image.
    faces = face_classifier.detectMultiScale(image_bw)
    
    for face in faces:
        x,y,w,h = face # Position of face 
        
        # Image setting
        cface_rgb = Image.fromarray(image_rgb[y:y+h, x:x+w])
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        image = cface_rgb
        image = ImageOps.fit(image, size, Image.ANTIALIAS)
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
        data[0] = normalized_image_array
        
        # Classification of whether a person is wearing a mask or not. 
        prediction = model.predict(data)
        
        # Setting color and text for each case.
        # Masked 
        if prediction[0][0] > prediction[0][1]:
            color = (255,102,51)
            cv2.rectangle(image_bgr,(x,y),(x+w,y+h),color,2)
            cv2.rectangle(image_bgr,(x-1, y-30),(x+w+1,y),color , cv2.FILLED)
            cv2.rectangle(image_bgr,(x-1, y+h),(x+w+1,y+h+30),color ,cv2.FILLED)
            
            cv2.putText(image_bgr,'Masked',(x+6,y-6),cv2.FONT_HERSHEY_DUPLEX,0.6,(255,255,255) ,2)
            cv2.putText(image_bgr,str(prediction[0][0]*100)+'%',(x+6,y+h+23),cv2.FONT_HERSHEY_DUPLEX,0.6,(255,255,255) ,2)
        
        # Non-Masked
        else:
            color = (46,2,209)
            cv2.rectangle(image_bgr,(x,y),(x+w,y+h),color,2)
            cv2.rectangle(image_bgr,(x-1, y-30),(x+w+1,y),color , cv2.FILLED)
            cv2.rectangle(image_bgr,(x-1, y+h),(x+w+1,y+h+30),color ,cv2.FILLED)
            
            cv2.putText(image_bgr,'Non-Masked',(x+6,y-6),cv2.FONT_HERSHEY_DUPLEX,0.6,(255,255,255) ,2)
            cv2.putText(image_bgr,str(prediction[0][1]*100)+'%',(x+6,y+h+23),cv2.FONT_HERSHEY_DUPLEX,0.6,(255,255,255) ,2)
    
    # Show window 
    cv2.imshow('Mask Detection',image_bgr)
    cv2.waitKey(1)
    